Exercise: Configuring Language Models and Prompts

Objective
To practice configuring language models (Cohere, Groq) and creating prompt templates to generate a story.

Create actor_picker Function:
Write a function named actor_picker() that randomly returns an actor's name. You can use Python's random generator to select an actor.

In [10]:
import os
import configparser

from langchain_groq import ChatGroq
from langchain_cohere import ChatCohere

from langchain_core.messages import HumanMessage, SystemMessage

config = configparser.ConfigParser()
config.read('../config.ini')
groq = config['groq']
cohere = config['cohere']

os.environ['GROQ_API_KEY'] = groq.get('GROQ_API_KEY')
os.environ['COHERE_API_KEY'] = cohere.get('COHERE_API_KEY')

In [11]:
import random

def actor_picker():
    # List of actor names
    actors = ["Shahrukh khan","Sanjay Datt","akshay kumar","paresh rawal","nana patekar"]
    
    # Randomly select and return an actor's name
    return random.choice(actors)

# Example usage
print(actor_picker())


paresh rawal


Create location_picker Function:
Write a function named location_picker() that randomly returns the name of a location.


In [12]:
import random

def location_picker():
    # List of locations from Japan and Maharashtra, India
    locations = [
        "Tokyo, Japan", "Kyoto, Japan", "Osaka, Japan", "Mount Fuji, Japan", 
        "Hokkaido, Japan", "Nara, Japan", "Okinawa, Japan", "Kobe, Japan",
        "Mumbai, Maharashtra, India", "Pune, Maharashtra, India", 
        "Nagpur, Maharashtra, India", "Nashik, Maharashtra, India", 
        "Aurangabad, Maharashtra, India", "Kolhapur, Maharashtra, India", 
        "Shirdi, Maharashtra, India", "Lonavala, Maharashtra, India"
    ]
    
    # Randomly select and return a location
    return random.choice(locations)

# Example usage
print(location_picker())


Mount Fuji, Japan


Create theme_picker Function:
Write a function named theme_picker() that randomly returns a theme.

In [13]:
import random

def theme_picker():
    # List of themes
    themes = [
        "Adventure", "Romance", "Mystery", "Horror", "Sci-Fi", "Fantasy", 
        "Comedy", "Drama", "Thriller", "Historical", "Superhero"
    ]
    
    # Randomly select and return a theme
    return random.choice(themes)

# Example usage
print(theme_picker())


Mystery


Configure and Use Cohere Model:
Set up the Cohere model using ChatCohere.
Create a prompt template that generates a story centered around the chosen Actor, Location, and Theme using the Cohere model.
Print the detailed story output from the model.

In [14]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate #import the Class for creating a prompt

parser = StrOutputParser()
actor = actor_picker() 
location = location_picker()
theme = theme_picker()

system_template = "generates a story centered around the chosen Actor, Location, and Theme.Please provide a {responseType} response." 

input = f"Create a story where {actor} is the main character in {location}, and the story follows a {theme} theme."
# Create prompt template instance.
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", system_template),
        ("user", input)
    ]
)

# prompt Template also implements runnable and can be easily chained.
model = ChatCohere(model="command-r-plus")
chain = prompt_template | model | parser

chain.invoke({"responseType":"brief"})

'Akshay, a renowned detective known for his sharp intellect and unconventional methods, arrived in Pune, Maharashtra, on a warm summer day. He had received an anonymous tip about a mysterious case that would challenge his skills and take him on a thrilling journey through the heart of the city.\n\nAs he stepped out of the train station, the bustling streets of Pune welcomed him. He made his way to a quaint café in the old part of town, where he had been instructed to wait for further instructions. As he sipped his chai, a young woman approached him, her eyes filled with worry. "Are you Detective Akshay Kumar?" she asked in a hushed tone. "Please, I need your help. My sister has gone missing, and I fear she may be in terrible danger."\n\nAkshay\'s curiosity was piqued. He assured the woman, named Maya, that he would do everything in his power to find her sister. With Maya by his side, he set off to explore the mysterious underbelly of Pune, uncovering clues and piecing together the puzz

Configure and Use Groq Model:
Set up the Groq model using ChatGroq.
Create a prompt template to generate a story using the Groq model.
Print the detailed response from the model.

In [15]:

model = ChatGroq(model="llama3-8b-8192")
chain = prompt_template | model | parser

chain.invoke({"responseType":"brief"})

'Title: The Pune Enigma\n\nAkshay Kumar, a renowned private investigator, stepped off the sweltering Pune streets and into the dimly lit office of his agency, Kumar Detective Services. The scent of fresh coffee and old books wafted through the air as he settled into his worn leather chair.\n\nThe phone on his desk jolted him out of his thoughts, and he answered it with a curt "Kumar speaking."\n\n"Ah, Akshay, I need your help," a familiar voice whispered on the other end. It was his old friend and fellow detective, Rohan.\n\n"What\'s the case, Rohan?" Akshay asked, his curiosity piqued.\n\n"It\'s a wealthy businessman, Ramesh Desai, who\'s gone missing in Pune. His wife, Nalini, is frantic. She thinks something\'s amiss, and the police are stumped. I\'ve managed to get her to agree to meet with you, hoping your unique skills can uncover what\'s happened to her husband."\n\nAkshay\'s eyes narrowed as he scribbled some notes on a pad. "I\'ll take the case. When and where?"\n\nThe meeting

Add InMemoryChatMessageHistory to store conversation:
Use 'InMemoryChatMessageHistory', to store history and allow user customize the story.
You can pass the story created with prompt saying 'change the given story according to user input'.

In [16]:
# import the chat history classes
from langchain_core.chat_history import (
    BaseChatMessageHistory,
    InMemoryChatMessageHistory,
)
import asyncio # library for writing code that interacts with DB, network calls etc. 

#Create a store in memory
store = InMemoryChatMessageHistory()


# Lets define a function that gets messages from store
async def getMessage():
    await asyncio.sleep(2) # this will mimic a read from DB
    print("Messages retrieved from DB")
    return await store.aget_messages()

# Add the initial messages to the store
store.add_message(HumanMessage(input))

# Retrieve the stored messages 
messages = await getMessage()

# Model invocation to generate a story based on the prompt
response = model.invoke(messages)  # Call the model with the messages
print("Generated Story:", response.content)  # Display the generated story

# Add the model's response to the store
store.add_message(SystemMessage(response.content))

Messages retrieved from DB
Generated Story: Title: The Pune Puzzle

Akshay Kumar, a renowned detective from Mumbai, had been summoned to Pune, the cultural capital of Maharashtra, to solve a baffling mystery. The case involved the disappearance of a wealthy businessman, Rohan Desai, who had gone missing from his luxurious bungalow in the upscale Koregaon Park area.

As Akshay arrived in Pune, he was greeted by the city's Deputy Commissioner of Police, who briefed him on the case. Rohan's wife, Priya, was frantic with worry, and the police were struggling to find any leads. Akshay was tasked with unraveling the mystery and finding Rohan before it was too late.

Akshay began his investigation by interviewing Priya and the Desai family members. He noticed that they seemed nervous and agitated, and he sensed that they might be hiding something. He decided to dig deeper and started by examining Rohan's office and personal workspace.

After a thorough search, Akshay discovered a cryptic mess

In [18]:
# allow the user to customize the story

user_input = "emotional ending in the climax"
user_input1 = f"change the given story according to user input {user_input}"

# Add the user's customization request to the store
store.add_message(HumanMessage(user_input1))

# Retrieve updated messages after user input
messages = await getMessage()

# Display all stored messages
print("Conversation History:", messages)

# Call the model again with the updated conversation including the user's input
response = model.invoke(messages)
print("Updated Story:", response.content)

# Add the updated model response to the store
store.add_message(SystemMessage(response.content))

Messages retrieved from DB
Conversation History: [HumanMessage(content='Create a story where akshay kumar is the main character in Pune, Maharashtra, India, and the story follows a Mystery theme.', additional_kwargs={}, response_metadata={}), SystemMessage(content='Title: The Pune Puzzle\n\nAkshay Kumar, a renowned detective from Mumbai, had been summoned to Pune, the cultural capital of Maharashtra, to solve a baffling mystery. The case involved the disappearance of a wealthy businessman, Rohan Desai, who had gone missing from his luxurious bungalow in the upscale Koregaon Park area.\n\nAs Akshay arrived in Pune, he was greeted by the city\'s Deputy Commissioner of Police, who briefed him on the case. Rohan\'s wife, Priya, was frantic with worry, and the police were struggling to find any leads. Akshay was tasked with unraveling the mystery and finding Rohan before it was too late.\n\nAkshay began his investigation by interviewing Priya and the Desai family members. He noticed that th